<img src="https://www.dbs.ie/images/default-source/logos/dbs-logo-2019-small.png" align = left/>

#  Open University Learning Analytics - Student Failure

Capstone Project

Claire Connaughton (10266499)

# Import Relevant Libraries

In [ ]:
import os
import pickle
import pydotplus
import numpy as np
import pandas as pd
from functools import reduce
from plotnine import *
import plotnine
plotnine.options.figure_size = (5.2,3.2)
import seaborn as sns
sns.set()
sns.set_style("white")
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.image as mpimg
from scipy import interp
from scipy.stats import skew, norm, probplot, boxcox, f_oneway
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler, RobustScaler
from sklearn.compose import make_column_transformer
from IPython.display import Image  
from sklearn.tree import export_graphviz
from six import StringIO
from collections import Counter
from pandas_profiling import ProfileReport
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler, RobustScaler
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split 
from sklearn.utils import resample 
from imblearn.over_sampling import SMOTE 
from sklearn.metrics import confusion_matrix 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold, cross_val_predict, train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, auc, accuracy_score, mean_absolute_error
from sklearn import metrics, tree
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC 
import xgboost as xgb
from sklearn.datasets import dump_svmlight_file
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

# Import dataset

In [ ]:
# Load the cleaned OULA dataset

try:
    data = pd.read_csv('oulad_cleaned.csv')
    print("The 'Cleaned OULA' dataset has {} samples with {} features each.".format(*data.shape))
except:
    print("Dataset could not be loaded. Is the dataset missing?")

In [ ]:
data['student_failed'].value_counts()

In [ ]:
# Display the number of students who passed or failed their course. 

g = sns.countplot(x ='student_failed', 
              data = data,
              palette='tab20',
              order = data['student_failed'].value_counts().index);

g.set_xticklabels(['Did not Fail','Failed'])
g.set_title('Student Outcome', fontsize = 18)


# Bivariate Analysis for Student Failure

***************************

# Demographic Details of Students who Failed

In [ ]:
g= sns.catplot('student_failed', col='code_module', col_wrap=4,
                data=data[data.code_module.notnull()],
                kind="count", height=3.5, aspect=.8, 
                palette= "tab20")
g.set_xticklabels(['Did not Fail','Failed'])
g.fig.subplots_adjust(top=0.9) 
g.fig.suptitle('Number of Fails per Module Type', fontsize = 18)

More students failed STEM courses.

In [ ]:
# Check if there is a gender difference in fails

g= sns.catplot('student_failed', col='gender', col_wrap=4,
                data=data[data.gender.notnull()],
                kind="count", height=3.5, aspect=.8, 
                palette= "tab20")
g.set_xticklabels(['Did not Fail','Failed'])
g.fig.subplots_adjust(top=0.6) 
g.fig.suptitle('Number of Fails per Gender', fontsize = 18)

Slightly more males failed their course. 

In [ ]:
# Display the number of fails per region

g= sns.catplot('student_failed', col='region', col_wrap=4,
                data=data[data.region.notnull()],
                kind="count", height=3.5, aspect=.8, 
                palette= "tab20")
g.set_xticklabels(['Did not Fail','Failed'])
g.fig.subplots_adjust(top=0.8) 
g.fig.suptitle('Number of Fails per Region', fontsize = 18)

West UK had slightly more fails than any other region.

In [ ]:
# Check if there is difference in highest education level among fails

g= sns.catplot('student_failed', col='highest_education', col_wrap=1,
                data=data[data.highest_education.notnull()],
                kind="count", height=3.5, aspect=.8, 
                palette= "tab20")
g.set_xticklabels(['Did not Fail','Failed'])
g.fig.subplots_adjust(top=0.8) 
g.fig.suptitle('Number of Fails per Education Level', fontsize = 18)

Students with a higher education qualification were the least likely to fail and students with the lowest educational attainment are the most likely to fail.

In [ ]:
# Check if there is difference in socio-economic status among fails

g= sns.catplot('student_failed', col='imd_band', col_wrap=4,
                data=data[data.imd_band.notnull()],
                kind="count", height=3.5, aspect=.8, 
                palette= "tab20")
g.set_xticklabels(['Did not Fail','Failed'])
g.fig.subplots_adjust(top=0.8) 
g.fig.suptitle('Number of Fails per Socio-Economic Class', fontsize = 18)

While middle class students were more likely to fail, the socioeconomically disadvantaged students had the second highest failure rate. The privileged group had the lowest failure rates.

In [ ]:
# Check if there is an age difference among fails

g= sns.catplot('student_failed', col='age_band', col_wrap=2,
                data=data[data.age_band.notnull()],
                kind="count", height=3.5, aspect=.8, 
                palette= "tab20")
g.set_xticklabels(['Did not Fail','Failed'])
g.fig.subplots_adjust(top=0.8) 
g.fig.suptitle('Number of Fails per Age Group', fontsize = 18)

Under 35s are the most likely to fail their course.

In [ ]:
# Check if there is a difference among student ability groups and fails

g= sns.catplot('student_failed', col='disability', col_wrap=2,
                data=data[data.disability.notnull()],
                kind="count", height=3.5, aspect=.8, 
                palette= "tab20")
g.set_xticklabels(['Did not Fail','Failed'])
g.fig.subplots_adjust(top=0.8) 
g.fig.suptitle('Number of Fails per Disability', fontsize = 18)

Disability does not seem to effect failure rate too much.

# The demographic profile of students who are more likely to fail is as follows:

1) Under 35s


2) Male


3) Having a disability


4) Studying a STEM course


5) From a lower socio-economic class


6) Holding lower than A-Level education



7) From West UK 

# Behavioural Details of Students who Dropout

In [ ]:
# Investigate the relationship between fails and the number of previous attempts

g = sns.catplot(x="student_failed", y="num_of_prev_attempts", palette= "tab20", kind="bar", data=data)
g.set_xticklabels(['Did not Fail','Failed'])
g.fig.subplots_adjust(top=0.9) 
g.fig.suptitle('Impact of Previous Attempts on Failure Rates', fontsize = 18)

There does not appear to be a strong link between failure and repeating a course. 

In [ ]:
# Investigate the relationship between failure and the number of studied credits

g = sns.catplot(x="student_failed", y="studied_credits", palette= "tab20", kind="bar", data=data)
g.set_xticklabels(['Did not Fail','Failed'])
g.fig.subplots_adjust(top=0.9) 
g.fig.suptitle('Impact of Studied Credits on Failure Rates', fontsize = 16)

The students who failed studied fewer credits. 

In [ ]:
# Investigate the relationship between failure and the Average number of clicks

g = sns.catplot(x="student_failed", y="AVG_click", palette= "tab20", kind= 'bar', data=data)
g.set_xticklabels(['Did not Fail','Failed'])
g.fig.subplots_adjust(top=0.9) 
g.fig.suptitle('Impact of AVG Clicks on Failure', fontsize = 18)

Students who failed had a lower number of average clicks on the Virtual Learning Environment.

In [ ]:
# Investigate the relationship between dropping out and the total clicks in the VLE

g = sns.catplot(x="student_failed", y="total_click", palette= "tab20", kind= 'bar', data=data)
g.set_xticklabels(['Passed', 'Failed'])
g.fig.subplots_adjust(top=0.9) 
g.fig.suptitle('Impact of Total Clicks on Failure', fontsize = 18)

The students who failed had considerably fewer total clicks than the students who completed the course.

In [ ]:
# Investigate the relationship between fails and late submission of assignments

sns.set_theme(style="ticks")
g = sns.catplot(x="student_failed", y="late_rate", palette= "tab20", kind= 'bar', data=data)
g.set_xticklabels(['Did not Fail','Failed'])
g.fig.subplots_adjust(top=0.9) 
g.fig.suptitle('Impact of Late Submission on Failure', fontsize = 18)


Students who failed were more likely to submit their assignment late.

In [ ]:
# Investigate the relationship between failure and the date unregistered

g= sns.catplot('student_failed', col='unreg_month', col_wrap=4,
                data=data[data.unreg_month.notnull()],
                kind="count", height=3.5, aspect=.8, 
                palette= "tab20")
g.set_xticklabels(['Did not Fail','Failed'])
g.fig.subplots_adjust(top=0.8) 
g.fig.suptitle('Impact of Date Unregistered on Failure', fontsize = 18)

 

Every student who dropped out before the end of the course in June, failed. Some students still failed in June. This means that if you completed the course you are more likely to pass. 

# The behaviour profile of students who failed is as follows:

1) Studied more credits so their workload was higher

2) Less inclined to interact with the VLE


3) More likely to submit assignments late


4) Completed the course.



**********************************************

# Multivariate Analysis for Student Failure

In [ ]:
# Use NumPy's corrcoef and seaborn's heatmap functions to plot the correlation matrix array as a heat map.

corr= data.corr()
top_corr_cols = corr.student_failed.sort_values(ascending=False).keys() 
top_corr = corr.loc[top_corr_cols, top_corr_cols]
dropSelf = np.zeros_like(top_corr)
dropSelf[np.triu_indices_from(dropSelf)] = True
plt.figure(figsize=(18, 10))
sns.heatmap(top_corr, cmap=sns.diverging_palette(220, 10, as_cmap=True), annot=True, fmt=".2f", mask=dropSelf)
sns.set(font_scale=1.5)

sns.set(font_scale=1.0)
cols = data.columns
g = sns.pairplot(data = data.loc[:, cols], hue='student_failed')
fig = g.fig 
fig.subplots_adjust(top=0.93, wspace=0.3)
t = fig.suptitle('Pairwise Plots by Student Failure', fontsize=24)

plt.show()

del corr, dropSelf, top_corr, g, fig, t

The multivariate analysis provides further confirmation that the students who failed were academically struggling and had lower engagement with the VLE.

# MODEL PREPARATION

Checking the Variance Inflation Factor for Each Variable

In [ ]:
# Find design matrix for linear regression model using 'rating' as response variable 
y, X = dmatrices('student_failed ~ code_module+code_presentation+reg_month+unreg_month+gender+region+highest_education+imd_band+age_band+num_of_prev_attempts+disability+studied_credits+total_click+weighted_score+late_rate+fail_rate+procrastination', data=data, return_type='dataframe')

#calculate VIF for each explanatory variable
vif = pd.DataFrame()
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['variable'] = X.columns

#view VIF for each explanatory variable 
vif

In [ ]:
# Using 5 as the minimum vif values i.e any independent variable 5 and above will have to be dropped
# From the results all independent variable are below 5
# Drop reg_month, unreg_month and code_module columns

data.drop(columns=['reg_month', 'unreg_month', 'code_module'], axis = 1, inplace = True)

In [ ]:
# Drop the ID column because it's unnecessary for model learning predictions

data.drop('id_student', axis = 1, inplace = True)

In [ ]:
#Declaring independent variable i.e X
#Declaring Target variable i.e y

x = data.drop(['student_failed', 'dropout'], axis = 1)
y = data['student_failed']

In [ ]:
# Set encoding and scaling instructions
column_transform = make_column_transformer(
    (OneHotEncoder(), ['code_presentation', 'gender', 'region', 'age_band', 'disability']),
    (OrdinalEncoder(), ['highest_education', 'imd_band']),
    (RobustScaler(), ['num_of_prev_attempts', 'studied_credits', 'total_click', 'late_rate'])
)

# Apply column transformer to features
X_encoded = column_transform.fit_transform(x)

In [ ]:
#Splitting data into train and test dataset

x_train, x_test, y_train, y_test = train_test_split(X_encoded, y , test_size = 0.2, random_state  = 365)

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=40) 

In [ ]:
# Set up SMOTE to treat imbalance in the target variable 
smote = SMOTE()  

In [ ]:
x_sm, y_sm =smote.fit_resample(x_train, y_train) 

print(x_train.shape, y_train.shape) 
print(x_sm.shape, y_sm.shape) 
sns.countplot(y_sm) 

*******************************

# MODELLING

***********************************

# Logistic Regression

In [ ]:
# Define logistic regression model function
def logistic_regression(x, y): 
    x_train, x_test, y_train, y_test =  train_test_split(x, y, test_size=0.20, random_state=40, stratify = y) 
    logreg_model = LogisticRegression() 
    logreg_model.fit(x_train, y_train) 

    pred_train = logreg_model.predict(x_train) 
    pred_test = logreg_model.predict(x_test) 
    cm_train = confusion_matrix(y_train, pred_train) 
    cm_test = confusion_matrix(y_test, pred_test)
    score = round(accuracy_score(y_test, pred_test), 3)
    cm1 = confusion_matrix(y_test, pred_test)
    sns.heatmap(cm1, annot=True, fmt=".0f")
    plt.xlabel('Predicted Values')
    plt.ylabel('Actual Values')
    plt.title('Accuracy Score: {0}'.format(score), size = 15)
    plt.show()
    print("Accuracy of Test Model : ",  logreg_model.score(x_test, y_test)) 
    print("Mean Absolute Error : ", mean_absolute_error(pred_test, y_test))
    print("Train Data Set") 
    print(metrics.classification_report(y_train,pred_train) ) 
    print("Test Data Set ") 
    print(metrics.classification_report(y_test,pred_test) ) 
    return  None 

In [ ]:
# Call logistic regression model

logistic_regression(x_sm, y_sm) 
warnings.filterwarnings('ignore')

# Decision Tree

In [ ]:
# Define decision tree model function

def decision_tree(x, y): 
    x_train, x_test, y_train, y_test =  train_test_split(x, y, test_size=0.20, random_state=40, stratify = y) 
    dt = DecisionTreeClassifier() 
    dt.fit(x_train, y_train) 

    pred_train = dt.predict(x_train) 
    pred_test = dt.predict(x_test) 
    confusion_matrix_train = confusion_matrix(y_train, pred_train) 
    confusion_matrix_test = confusion_matrix(y_test, pred_test)
    score = round(accuracy_score(y_test, pred_test), 3)
    cm1 = confusion_matrix(y_test, pred_test)
    sns.heatmap(cm1, annot=True, fmt=".0f")
    plt.xlabel('Predicted Values')
    plt.ylabel('Actual Values')
    plt.title('Accuracy Score: {0}'.format(score), size = 15)
    plt.show()
    print("Accuracy of Test Model : ",  dt.score(x_test, y_test)) 
    print("Mean Absolute Error : ", mean_absolute_error(pred_test, y_test))
    print("Train Data Set") 
    print(metrics.classification_report(y_train,pred_train) ) 
    print("Test Data Set ") 
    print(metrics.classification_report(y_test,pred_test) ) 
    return  None 

In [ ]:
# Call deicsion tree model

decision_tree(x_sm, y_sm) 
warnings.filterwarnings('ignore')

# Random Forest

In [ ]:
# Define random forest model function

def random_forest(x, y): 
    x_train, x_test, y_train, y_test =  train_test_split(x, y, test_size=0.20, random_state=40, stratify = y) 
    rf = RandomForestClassifier() 
    rf.fit(x_train, y_train) 

    pred_train = rf.predict(x_train) 
    pred_test = rf.predict(x_test) 
    confusion_matrix_train = confusion_matrix(y_train, pred_train) 
    confusion_matrix_test = confusion_matrix(y_test, pred_test)
    score = round(accuracy_score(y_test, pred_test), 3)
    cm1 = confusion_matrix(y_test, pred_test)
    sns.heatmap(cm1, annot=True, fmt=".0f")
    plt.xlabel('Predicted Values')
    plt.ylabel('Actual Values')
    plt.title('Accuracy Score: {0}'.format(score), size = 15)
    plt.show()
    print("Accuracy of Test Model : ",  rf.score(x_test, y_test)) 
    print("Mean Absolute Error : ", mean_absolute_error(pred_test, y_test))
    print("Train Data Set") 
    print(metrics.classification_report(y_train,pred_train) ) 
    print("Test Data Set ") 
    print(metrics.classification_report(y_test,pred_test) ) 
    return  None 

In [ ]:
# Call random forest model

random_forest(x_sm, y_sm) 
warnings.filterwarnings('ignore')

# Support Vector Machine

In [ ]:
# Define Support Vector Machine model function

def support_vector_machine(x, y): 
    x_train, x_test, y_train, y_test =  train_test_split(x, y, test_size=0.25, random_state=40, stratify = y) 
    svm_model_linear = SVC(kernel = 'linear', C = 1).fit(x_train, y_train) 
    svm_predictions = svm_model_linear.predict(x_test)

    pred_train = svm_model_linear.predict(x_train) 
    pred_test = svm_model_linear.predict(x_test) 
    confusion_matrix_train = confusion_matrix(y_train, pred_train) 
    confusion_matrix_test = confusion_matrix(y_test, pred_test)
    score = round(accuracy_score(y_test, pred_test), 3)
    cm1 = confusion_matrix(y_test, pred_test)
    sns.heatmap(cm1, annot=True, fmt=".0f")
    plt.xlabel('Predicted Values')
    plt.ylabel('Actual Values')
    plt.title('Accuracy Score: {0}'.format(score), size = 15)
    plt.show()
    print("Accuracy of Test Model : ",  svm_model_linear.score(x_test, y_test)) 
    print("Mean Absolute Error : ", mean_absolute_error(pred_test, y_test))
    print("Train Data Set") 
    print(metrics.classification_report(y_train,pred_train) ) 
    print("Test Data Set ") 
    print(metrics.classification_report(y_test,pred_test) ) 
    return  None 

In [ ]:
# Call Support Vector Machine model
support_vector_machine(x_sm, y_sm) 
warnings.filterwarnings('ignore')

# XGBoost

In [ ]:
# Train and test split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.25)

In [ ]:
# Set up xgBoost matrix
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [ ]:
dump_svmlight_file(X_train, y_train, 'dtrain.svm', zero_based=True)
dump_svmlight_file(X_test, y_test, 'dtest.svm', zero_based=True)
dtrain_svm = xgb.DMatrix('dtrain.svm')
dtest_svm = xgb.DMatrix('dtest.svm')

In [ ]:
# Set parameters
param = {
    'max_depth': 3,  # the maximum depth of each tree
    'eta': 0.3,  # the training step for each iteration
    'silent': 1,  # logging mode - quiet
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 3}  # the number of classes that exist in this datset
num_round = 50  # the number of training iterations

In [ ]:
# training and testing - numpy matrices
bst = xgb.train(param, dtrain, num_round)
preds = bst.predict(dtest)
# extracting most confident predictions
best_preds = np.asarray([np.argmax(line) for line in preds])
print ("Numpy array precision:", precision_score(y_test, best_preds, average='macro'))

In [ ]:
# Print model metrics 
print(accuracy_score(y_test, best_preds))
print(recall_score(y_test, best_preds, average='macro'))
print(f1_score(y_test, best_preds, average='macro'))
print(precision_score(y_test, best_preds, average='macro'))

In [ ]:
# Set up confusion matrix
score = round(accuracy_score(y_test, best_preds), 3)
cm1 = confusion_matrix(y_test, best_preds)
sns.heatmap(cm1, annot=True, fmt=".0f")
plt.xlabel('Predicted Values')
plt.ylabel('Actual Values')
plt.title('Accuracy Score: {0}'.format(score), size = 15)
plt.show()
print(metrics.classification_report(y_test,best_preds) )

# Comparing Model Performance

In [ ]:
# Plotting Bar Chart for Accuracy of different classifiers

plt.figure(figsize=(12, 3))
model_accuracies = [0.83, 0.79, 0.74, 0.64, 0.64]
model_names = ['RandomForest','XGBoost','Decision Tree', 'Logistic Regression','Support Vector Machine']
g= sns.barplot(x=model_names, y=model_accuracies, color='grey');

patch_h = []    
for patch in g.patches:
    reading = patch.get_height()
    patch_h.append(reading)

idx_tallest = np.argmax(patch_h)   

g.patches[idx_tallest].set_facecolor('seagreen')  
g.set_title('Model Accuracy for Student Failure', fontsize = 18)


# Determine Feature Importance

In [ ]:
# Extract feature importance
model = ExtraTreesRegressor()
model.fit(X_encoded, y)

In [ ]:
# Retrieve column names
X_encoded= pd.DataFrame(X_encoded) 
x_names= X_encoded.iloc[:,:17]
x_names=pd.DataFrame(x_names)
model = ExtraTreesRegressor()
model.fit(x_names,y)

In [ ]:
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=x.columns)
feat_importances.nlargest(8).plot(kind='barh')
plt.show()

Whether a student procrastinates, fails certain assessments, has late submissions and lower education increases the liklihood that they will fail their course. 

# Attempt to improve model accuracy with PCA

In [ ]:
# Set up PCA
pca = PCA()
X_train = pca.fit_transform(X_train)
pca.explained_variance_ratio_

In [ ]:
# Determine the optimal number of principal components
pca = sklearnPCA().fit(x_sm)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')

About 13 components explain 95% of the variance in student failure

In [ ]:
# Run logistic regression with PCA

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

print('Logistic Regression accuracy score with the first 13 features: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

In [ ]:
dt = DecisionTreeClassifier() 
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
print('Decision Tree accuracy score with the first 15 features: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

In [ ]:
rf=RandomForestClassifier() 
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print('Random Forest accuracy score with the first 15 features: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

PCA improves predictive accuracy for logistic regression but not the other models.

# END